In [1]:
import pandas as pd
import re, time, glob, sys, collections
import scattertext as st
from IPython.display import IFrame
from IPython.core.display import display, HTML
import ipywidgets as widgets
import seaborn as sns
display(HTML("<style>.container { width:98% !important; }</style>"))
import spacy
import scattertext as st
sys.path.append('../scripts/')
from paper_info import Paper
%matplotlib inline

In [2]:
def build_data_frame():
    data = []
    for fn in glob.glob('../data/*/*/final/*/*_metadata.txt'):
        paper = Paper(fn)
        d = {'abstract':paper.abstract,
             'title':paper.escaped_title(),
             'n_authors':len(paper.authors)}
        for i, author in enumerate(paper.authors):
            author_place = ('0' if i < 9 else '') + str(i+1)
            d['%s_first_name'%author_place] = author.first
            d['%s_last_name'%author_place] = author.last
            d['%s_email_domain'%author_place] = author.email.split('@')[1]
        d['venue'] = fn.split('/')[2]
        d['meta'] = fn.split('/')[2] + ': ' + str(paper)
        data.append(d)
    return pd.DataFrame(data)
df = build_data_frame()

In [3]:
nlp = spacy.en.English()

In [4]:
df['parse'] = (df['title'] + '.\n\n' + df['abstract']).apply(nlp)

In [46]:
for venue in df['venue'].unique():
    df['category'] = df['venue'].apply(lambda x: venue if x == venue else 'Other venue')
    html = st.produce_scattertext_explorer(st.CorpusFromParsedDocuments(df, category_col='category', parsed_col='parse').build(),
                                           category=venue,
                                           category_name=venue,
                                           not_category_name='All Others',
                                           width_in_pixels=1000,
                                           minimum_term_frequency=4,
                                           pmi_threshold_coefficient=6,
                                           metadata=venue_view['meta'])
    file_name = venue+'.html'
    with open(file_name, 'wb') as fn:
        fn.write(html.encode('utf-8'))

In [62]:
with open('selector.html', 'w') as fn:
    contents = '''
<html><body>
<h2 style="font-family: Helvetica, Arial, Sans-Serif">See how language use differed among ACL 2017 tracks and workshops</h2>
<div style="font-family: Helvetica, Arial, Sans-Serif">
For an introduction to language analysis using Scattertext, please see:
<ul>
<li>The ACL 2017 Demo paper <a href="https://arxiv.org/abs/1703.00565">here</a>.</li>
<li>The Youtube video of the 2017 PyData <a href="https://www.youtube.com/watch?v=H7X9CA2pWKoa">talk</a>.</li>
<li>Scattertext's Github <a href="https://github.com/JasonKessler/scattertext">repo</a>.</li>
<li>The source notebook for this analysis <a href="https://github.com/JasonKessler/scattertext">repo</a>.</li>
</ul>
Cite as: 
<pre>Jason S. Kessler. Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ. ACL System Demonstrations. Vancouver, BC. 2017.</pre>
</div> 
<form><select id=selector" onchange="document.getElementById('myiframe').src = this.options[this.selectedIndex].value">
<option>Select a track or workshop:</option>
''' + ''.join(['<option value="%s.html">%s</option>\n'%(venue,venue) for venue in df['venue'].unique()]) + '''
</select></form><iframe name="myiframe" id="myiframe" width=1200 height=800 src=""></iframe></body></html>
'''
    fn.write(contents)

In [63]:
%%HTML
<iframe width=100% height=800 name="iframe" src="selector.html"></iframe>